In [279]:
import pandas as pd
import numpy as np

In [280]:
application_record = pd.read_csv('application_record.csv', index_col=0)
application_record.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
ID,,,,,,,,,,,,,,,,,
5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [281]:
credit_record = pd.read_csv('credit_record.csv')
credit_record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [282]:
# Pivoting the original DataFrame to get status counts as columns
pivoted_df = credit_record.pivot_table(index='ID', columns='STATUS', aggfunc='size', fill_value=None).reset_index()

pivoted_df.head()

STATUS,ID,0,1,2,3,4,5,C,X
0,5001711,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,5001712,10.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
2,5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
3,5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
4,5001715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0


In [283]:
merged_df = pd.merge(application_record, pivoted_df, how="right", on="ID")

merged_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,OCCUPATION_TYPE,CNT_FAM_MEMBERS,0,1,2,3,4,5,C,X
0,5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
2,5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
3,5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
4,5001715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0


In [284]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45985 entries, 0 to 45984
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   45985 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  float64
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  float64
 11  DAYS_EMPLOYED        36457 non-null  float64
 12  FLAG_MOBIL           36457 non-null  float64
 13  FLAG_WORK_PHONE      36457 non-null  float64
 14  FLAG_PHONE           36457 non-null  float64
 15  FLAG_EMAIL           36457 non-null 

In [285]:
columns = merged_df.iloc[:, 17:].columns
merged_df[columns] = merged_df[columns].astype('Int64')

In [286]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45985 entries, 0 to 45984
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   45985 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  float64
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  float64
 11  DAYS_EMPLOYED        36457 non-null  float64
 12  FLAG_MOBIL           36457 non-null  float64
 13  FLAG_WORK_PHONE      36457 non-null  float64
 14  FLAG_PHONE           36457 non-null  float64
 15  FLAG_EMAIL           36457 non-null 

In [287]:
merged_df.describe()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,0,1,2,3,4,5,C,X
count,4.598500e+04,36457.000000,3.645700e+04,36457.000000,36457.000000,36457.0,36457.000000,36457.000000,36457.000000,36457.0,39980.0,5201.0,597.0,268.0,190.0,195.0,22707.0,25005.0
mean,5.070164e+06,0.430315,1.866857e+05,-15975.173382,59262.935568,1.0,0.225526,0.294813,0.089722,2.198453,9.582791,2.132282,1.453936,1.19403,1.173684,8.682051,19.466728,8.367526
std,4.543364e+04,0.742367,1.017892e+05,4200.549944,137651.334859,0.0,0.417934,0.455965,0.285787,0.911686,8.123597,2.156698,0.96062,0.605562,0.587653,9.911018,13.569396,11.762083
min,5.001711e+06,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,5.026147e+06,0.000000,1.215000e+05,-19438.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.0,4.0,1.0,1.0,1.0,1.0,2.0,8.0,1.0
50%,5.065737e+06,0.000000,1.575000e+05,-15563.000000,-1552.000000,1.0,0.000000,0.000000,0.000000,2.0,7.0,1.0,1.0,1.0,1.0,5.0,17.0,3.0
75%,5.114024e+06,1.000000,2.250000e+05,-12462.000000,-408.000000,1.0,0.000000,1.000000,0.000000,3.0,12.0,2.0,2.0,1.0,1.0,11.0,29.0,11.0
max,5.150487e+06,19.000000,1.575000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.0,61.0,38.0,8.0,8.0,7.0,48.0,60.0,61.0


In [288]:
merged_df['DAYS_EMPLOYED'].sort_values(ascending = False).unique()

array([ 3.65243e+05, -1.70000e+01, -4.30000e+01, ..., -1.56610e+04,
       -1.57130e+04,          nan])

In [289]:
merged_df['DAYS_EMPLOYED'] = merged_df['DAYS_EMPLOYED'].apply(lambda x : 0 if x > 0 else -x)

merged_df['DAYS_BIRTH'] = merged_df['DAYS_BIRTH'] * -1

merged_df.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Property','CNT_CHILDREN':'Children','AMT_INCOME_TOTAL':'Income',
'NAME_INCOME_TYPE':'Income status', 'NAME_EDUCATION_TYPE':'Education','NAME_FAMILY_STATUS':'Marital', 'NAME_HOUSING_TYPE':'Housing', 'DAYS_BIRTH': 'Day since birth',
'DAYS_EMPLOYED' : 'Days employed', 'FLAG_MOBIL' : 'Mobile', 'FLAG_WORK_PHONE':'Work phone', 'FLAG_PHONE':'Phone', 'FLAG_EMAIL':'Email', 'OCCUPATION_TYPE':'Job',
'CNT_FAM_MEMBERS':'Family'},inplace=True)

merged_df.head()

,ID,Gender,Car,Property,Children,Income,Income status,Education,Marital,Housing,...,Job,Family,0,1,2,3,4,5,C,X
0,5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
1,5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,10,<NA>,<NA>,<NA>,<NA>,<NA>,9,<NA>
2,5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22
3,5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15
4,5001715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,60


In [290]:
merged_df.to_csv('merged_df.csv', index = False)